<!-- 1. All the genes to include are in ./shortgenes2design_full.tsv. Specifically the short genes are in ./selected_shortgenes.tsv. 
2. ppDesigner results for the short genes are ./shortgenes_PPD/probes_DpnII-filtered_ov20.ppd
3. For the non-short (or long) genes, we have two sources from which to select probes. We will use the overlapping source if there are not enough non-overlapping probes
    1. ../221021_WholeGenome_noOverlapProbes/probes_DpnII-filtered.ppd
    2. ../221010_WholeGenome_overlappingProbes/probes_DpnII-filtered.ppd
4. The probes are going to be 3-on 3-off
5. Amplification primers are going to be AP1V6 and AP2V6 (AP1V6 is a modified version of AP1V7) -->
1. sppDesigner run is here: /home/kkalhor/Codes/ppDesignerRuns/230725_hg38_overlap20/probes_DpnII-filtered.ppd
2. gene to barcode assignments are here: "./gene_assignments/best_run.tsv"

We are aiming for 50 probes per gene. For every gene, we make two subsets of probes: the larger, l-subset and the smaller, s-subset. The probes in each subset are mutually non-overlapping, but they will overlap across subsets. If the l-subset has more than ```n_max``` probes, we randomly sample ```n_max``` probes without replacement. Otherwise, if l-subset and s-subset together have more than ```n_max``` probes, the select all of l-subset and then top it off with random sampling from s-subset. If l-subset and s-subset together have less than ```n_max``` probes, select all of them and top off with random sampling. 

In [44]:
import os, re
import numpy as np, pandas as pd
import pickle
from matplotlib import pyplot as plt
rng = np.random.seed(seed=0)

n_max = 10 # maximum number of probes per gene
target_len = 152 # add A's to the 5' of the final probes to get them to this length
assign_df = pd.read_csv("./barcodes/240508_3on-5off-3col_Hamming2_network_panel.tsv", sep="\t", index_col=0).dropna()
empty_df = assign_df.loc[(assign_df['Gene name'].str.startswith('Empty'))]
assign_df = assign_df.loc[(~assign_df['Gene name'].str.startswith('Empty'))&(assign_df['Source']!='Human')]
allgenes = assign_df['Gene name'].unique()
display(assign_df)

,index,0,1,2,3,4,5,6,7,Gene name,Source
0,0,1,2,0,0,0,0,0,3,Sox9,Mouse
1,1,1,0,2,0,3,0,0,0,Dcn,Mouse
2,2,0,0,1,0,0,1,0,2,Pdgfra,Mouse
3,3,2,2,0,0,0,0,3,0,Thy1,Mouse
4,4,0,0,3,3,0,0,2,0,Cdh5,Mouse
...,...,...,...,...,...,...,...,...,...,...,...
309,914,2,0,0,2,0,1,0,0,Fosl1,Mouse
310,916,0,0,1,0,0,0,3,2,Fosl2,Mouse
311,918,2,0,1,0,0,0,1,0,Atf3,Mouse
312,924,0,0,2,0,0,2,0,1,Atf4,Mouse


In [45]:
# determine the probe is in CDS
import gtfparse, os, pandas as pd
annot_file = os.path.join('/home/xli/reference/mouse/mm39/gencode.vM35.annotation.gtf')
annot_df = gtfparse.read_gtf(annot_file)


INFO:root:Extracted GTF attributes: ['gene_id', 'gene_type', 'gene_name', 'level', 'mgi_id', 'havana_gene', 'transcript_id', 'transcript_type', 'transcript_name', 'transcript_support_level', 'tag', 'havana_transcript', 'exon_number', 'exon_id', 'protein_id', 'ccdsid', 'ont']


In [46]:

refseq_df = pd.DataFrame(data=annot_df, columns=annot_df.columns)
refseq_df = refseq_df.loc[(refseq_df['gene_name'].isin(assign_df['Gene name']))&
                            (refseq_df['feature']=='CDS')&
                            (refseq_df['tag'].str.contains('Ensembl_canonical'))] # only design probes targeting canonical transcripts
refseq_df = refseq_df.groupby(['gene_name','feature','strand']).agg({'start': 'min', 'end': 'max'}).reset_index()
# add ncRNA gene: Morrbid
refseq_df = pd.concat([refseq_df,pd.DataFrame({'gene_name': 'Morrbid', 'feature': 'CDS', 'strand': '-', 'start': 128020239, 'end': 128271286}, index=[0])])

In [47]:
set(assign_df['Gene name']) - set(refseq_df['gene_name'])

set()

In [51]:
def readPPD(file):
    ppd = pd.read_csv(file, sep="\t")
    ppd['gene_name'] = ppd['0'].str.split('_').str[0]
    ppd[['chr', 'exon_start', 'exon_end']] = ppd['1'].str.extract(r"(chr\S+):(\d+)-(\d+)")
    ppd['probe_center'] = ppd['exon_start'].astype(int) + ppd['3'].astype(int)
    return ppd
all_ppd = readPPD("/home/xli/workspaces/dartfish/ProbeDesign/Codes/ppDesignerRuns/240724_mm39_splintR_overlap20/probes_DpnII-filtered.ppd")
all_ppd = all_ppd.query("gene_name in @allgenes")
all_ppd
# # set(short_ppd['gene']).difference(shortgenes)

# overlap_ppd = readPPD("../221010_WholeGenome_overlappingProbes/probes_DpnII-filtered.ppd")
# noovrlp_ppd = readPPD("../221021_WholeGenome_noOverlapProbes/probes_DpnII-filtered.ppd")
# overlap_ppd = overlap_ppd.query('gene_name in @longgenes')
# noovrlp_ppd = noovrlp_ppd.query('gene_name in @longgenes')
# # overlap_ppd

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,gene_name,chr,exon_start,exon_end,probe_center
7019,Il17a_chr1_1,chr1:020801129-20801212,0,65.0,65.0,GCAAACATGAGTCCA,50.0,GGGAGAGCTTCATCT,51.0,2.370986,W,GCAAACATGAGTCCAGGGAGAGCTTCATCT,1.2,1s,Il17a,chr1,020801129,20801212,20801194
7020,Il17a_chr1_2,chr1:020802320-20802531,0,69.0,69.0,ATCATCCCTCAAAGCT,51.0,CAGCGTGTCCAAACACT,58.0,2.416665,W,ATCATCCCTCAAAGCTCAGCGTGTCCAAACACT,0.5,2s,Il17a,chr1,020802320,20802531,20802389
7021,Il17a_chr1_2,chr1:020802320-20802531,0,93.0,93.0,AAACACTGAGGCCA,50.0,AGGACTTCCTCCAGAATGT,58.0,2.437995,W,AAACACTGAGGCCAAGGACTTCCTCCAGAATGT,0.5,2s,Il17a,chr1,020802320,20802531,20802413
7022,Il17a_chr1_2,chr1:020802320-20802531,0,112.0,112.0,GACTTCCTCCAGAATGT,53.0,GAAGGTCAACCTCAAA,50.0,2.539507,W,GACTTCCTCCAGAATGTGAAGGTCAACCTCAAA,0.5,2s,Il17a,chr1,020802320,20802531,20802432
7023,Il17a_chr1_2,chr1:020802320-20802531,0,130.0,130.0,AAGGTCAACCTCAAAGT,53.0,CTTTAACTCCCTTGGC,51.0,2.531687,W,AAGGTCAACCTCAAAGTCTTTAACTCCCTTGGC,0.5,2s,Il17a,chr1,020802320,20802531,20802450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1471967,Kdm5d_chrY_26,chrY:000942824-946316,0,1052.0,1052.0,TTGCTGTGGGACTCAA,55.0,CAGCATTTGTTTCTGAA,50.0,2.611300,W,TTGCTGTGGGACTCAACAGCATTTGTTTCTGAA,0.0,14s,Kdm5d,chrY,000942824,946316,943876
1471968,Kdm5d_chrY_26,chrY:000942824-946316,0,1278.0,1278.0,ACCACTGCCCAACT,53.0,CTATGCTTCTTGATACATG,51.0,2.552558,W,accactgcccaactCTATGCTTCTTGATACATG,0.0,14s,Kdm5d,chrY,000942824,946316,944102
1471969,Kdm5d_chrY_26,chrY:000942824-946316,0,3086.0,3086.0,TGGAGATGAATAAGTTGT,50.0,AAGGGTTGAGGCTAA,50.0,2.485002,W,tggagatgaataagttgtaagggttgaggctaa,0.0,14s,Kdm5d,chrY,000942824,946316,945910
1471970,Kdm5d_chrY_26,chrY:000942824-946316,0,3233.0,3233.0,TCCATGAGATAAGGCA,51.0,ACTGTAAGGCATTTTCT,50.0,2.493049,W,tccatgagataaggcaactgtaaggcattttct,0.0,14s,Kdm5d,chrY,000942824,946316,946057


In [52]:
all_ppd = all_ppd.loc[all_ppd['gene_name'].isin(refseq_df['gene_name'])]

# Annotate probes to CDS, 5'UTR or 3'UTR, considering the strand
Annotation = []
for i, row in all_ppd.iterrows():
    gene = row['gene_name']
    gene_df = refseq_df.loc[refseq_df['gene_name']==gene]
    if len(gene_df)==0:
        Annotation.append('NA')
    else:
        if gene_df['strand'].values[0]=='+':
            if row['probe_center']<gene_df['start'].values[0]:
                Annotation.append("5'UTR")
            elif row['probe_center']>gene_df['end'].values[0]:
                Annotation.append("3'UTR")
            else:
                Annotation.append("CDS")
        else:
            if row['probe_center']<gene_df['start'].values[0]:
                Annotation.append("3'UTR")
            elif row['probe_center']>gene_df['end'].values[0]:
                Annotation.append("5'UTR")
            else:
                Annotation.append("CDS")
all_ppd['Annotation'] = Annotation

In [40]:
""" Find the l-subset and s-subset for each gene"""
temp = []
for g in set(allgenes) - set(['Ccl27a','Cbx3','Bnip3']):
    ppd_g = all_ppd.query("gene_name == @g").copy()
    ppd_g = ppd_g.sort_values('probe_center')
    ppd_g['dist2prev'] = np.inf
    centers = ppd_g['probe_center'].values
    left_lens = ppd_g['5'].str.len().values # length of the 5' targets
    right_lens = ppd_g['7'].str.len().values # length of the 3' targets
    ppd_g.iloc[1:, ppd_g.columns.get_loc('dist2prev')] = centers[1:] - centers[0:-1]
    ppd_g['backreach'] = 0 # minimum distance between two probe to get no overlap
    ppd_g.iloc[1:, ppd_g.columns.get_loc('backreach')] = right_lens[0:-1] + left_lens[1:]
    realsubset = ['l']
    for i, (ind, (d, r)) in enumerate(ppd_g[['dist2prev', 'backreach']].iterrows()):
        if i == 0:
            continue
        if r > d: # current probe overlaps with the previous probe
            if realsubset[i-1] == 'l': # previous probe is in l-subset
                realsubset.append('s')
                continue
        # if we're here it means that reach < distance, the previous probe was in s-subset
        realsubset.append('l')
    ppd_g['subset'] = realsubset
    temp.append(ppd_g)    
all_ppd = pd.concat(temp)
all_ppd

ValueError: Must have equal len keys and value when setting with an iterable

### Preparing the decoding sequences
5 probes have DpnII cut site in them. We replace them with new probes.

In [54]:
def getDcName(rnd, cl):
    if cl == 0:
        raise ValueError('Value 0 not accepted for a probe color')
    fl_cl = {'1' : '488', '2' : 'cy3', '3' : 'cy5'}
    return "dc{0}-{1}".format(rnd, fl_cl[str(cl)])

def reverse_complement(dna):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A'}
    return ''.join([complement[base] for base in dna[::-1]])

dcProbes_File = './Decoding_Probes.xlsx'
dpnii_site = 'GATC'
dcProbes_df = pd.read_excel(dcProbes_File, sheet_name='all decoding probes', index_col=0)
display(dcProbes_df.head())
display(dcProbes_df.loc[dcProbes_df['probe sequence'].str.contains(dpnii_site)])

dcProbes_df = dcProbes_df.loc[~dcProbes_df['probe sequence'].str.contains(dpnii_site)]
dcProbes_df = dcProbes_df.set_index(dcProbes_df.index.str.split('_').str[0])

,probe sequence,decoder sequence
Name,,
dc0-488,TGTATCGCGCTCGATTGGCA,TGCCAATCGAGCGCGATACA
dc0-cy3,CGTATCGGTAGTCGCAACGC,GCGTTGCGACTACCGATACG
dc0-cy5,ACGCTACGGAGTACGCCACT,AGTGGCGTACTCCGTAGCGT
dc1-488,TCTTGCGTGCGATACGGAGT,ACTCCGTATCGCACGCAAGA
dc1-cy3,AACGGTATTCGGTCGTCATC,GATGACGACCGAATACCGTT


,probe sequence,decoder sequence
Name,,
dc3-cy3,CGTTTGATCGTTCGACCGAG,CTCGGTCGAACGATCAAACG
dc4-cy5,CAGGGATCGGTCGAGTACGC,GCGTACTCGACCGATCCCTG
dc6-488,CACGCTTACGATCCCGCTAT,ATAGCGGGATCGTAAGCGTG
dc7-488,TGATCCTCCGCGAATCCATG,CATGGATTCGCGGAGGATCA
dc10-cy5,CCACAGATCGTGACTATCGG,CCGATAGTCACGATCTGTGG
dc4-ATTO647N,CAGGGATCGGTCGAGTACGC,GCGTACTCGACCGATCCCTG
dc10-ATTO647N,CCACAGATCGTGACTATCGG,CCGATAGTCACGATCTGTGG


In [55]:
barcodes_df = assign_df.copy() #assign_df.drop('gene', axis=1).copy()
barcodes_seq = pd.DataFrame()
for i in barcodes_df.index:
    nonZeros_bc = barcodes_df.drop(['index','Gene name','Source'], axis=1).loc[i, barcodes_df.loc[i] != 0].to_dict()
    nonZero_rnds = list(nonZeros_bc.keys())
    nonZeros_seq = {rnd : dcProbes_df['probe sequence'].loc[getDcName(rnd, nonZeros_bc[rnd])]
                    for rnd in nonZero_rnds}
    bc_str = "".join(barcodes_df.drop(['index','Gene name','Source'], axis=1).loc[i].astype(str))
    
    thisBarcodes_seq = pd.DataFrame({'barcode' : bc_str, 'Gene name' : barcodes_df.loc[i, 'Gene name']},
                                    index = [i]) # a df with one row
    for j, rnd in enumerate(nonZeros_seq):
        thisBarcodes_seq.loc[:, j] = nonZeros_seq[rnd] # adding dcProbes to the df
    barcodes_seq = pd.concat([barcodes_seq, thisBarcodes_seq])
    
print(barcodes_seq.shape)
barcodes_seq

(311, 5)


,barcode,Gene name,0,1,2
0,12000003,Sox9,TGTATCGCGCTCGATTGGCA,AACGGTATTCGGTCGTCATC,GCTCAGCCGGACGAGTAGAT
1,10203000,Dcn,TGTATCGCGCTCGATTGGCA,CTACTTCGTCGCGTCAGACC,ACTCTACCGGCAATCGCGTC
2,00100102,Pdgfra,AGAACTTGCGCGGATACACG,GAGTGTCGCGCAACTTAGCG,GCCACATCGACTCGGTCTAT
3,22000030,Thy1,CGTATCGGTAGTCGCAACGC,AACGGTATTCGGTCGTCATC,CTCTCGTAGCGTGCGATGAG
4,00330020,Cdh5,GACGAACGGTCGAGATTTAC,AACTGCGACCGTCGGCTTAC,TCGTAACCCGTGCGAAGTGC
...,...,...,...,...,...
309,20020100,Fosl1,CGTATCGGTAGTCGCAACGC,TCGTACTTCGACGGCACTCA,GAGTGTCGCGCAACTTAGCG
310,00100032,Fosl2,AGAACTTGCGCGGATACACG,CTCTCGTAGCGTGCGATGAG,GCCACATCGACTCGGTCTAT
311,20100010,Atf3,CGTATCGGTAGTCGCAACGC,AGAACTTGCGCGGATACACG,CTTGCGGCGACAGTCGAACA
312,00200201,Atf4,CTACTTCGTCGCGTCAGACC,ACGTCTGCGTACCGGCTTAG,TTAGGTCGCCTACCGACTGC


In [57]:
barcodes_df.loc[barcodes_df['Gene name']=='Junb']

,index,0,1,2,3,4,5,6,7,Gene name,Source
306,906,0,1,0,3,0,0,2,0,Junb,Mouse


## Synthesizing the barcode sequence
We want to make sure to have DpnII free probes. So we'll have to have a combination of barcodes for each probe that doesn't give us GATC. Since the left and right arms, common insert and the decoding probes don't have native DpnII sites in them, potential problem could arise at junctions. 

For each barcode, we synthesize the probe with the following recipe:
5' -> AP1 - left arm - barcode i1 - anchor probe - barcode i2 - barcode i3 - right arm - AP2 -> 3'  
i1...i3 are a permutation of {0, 1, 2} that give us a DpnII free full probe (excluding the native DpnII cut site of AP2)

In addition, we want all probes to be at least 160 bases long (max is 162). I'll add a number of As before the common insert


In [58]:
# commonInsert = 'CTTCAGCTTCCCGATATCCGACGG' # 61C
anchorProbe = 'CTTCAGCTTCCCGATATCCG' # 55.1C
rcaPrimer = reverse_complement('GATATCGGGAAGCTGAAG') # 49.6C
print('anchorProbe = {0}, rcaPrimer = {1}'.format(len(anchorProbe), len(rcaPrimer)))
# AP1V6 = 'GTCATATCGGTCACTGTT'  # AP1 directly goes on the probes. Order pAP1V61U G*G*GTCATATCGGTCACTGTU for amplification
# AP2V6 = 'GGGTAGTGTGTATCCTGATC' # AP2 will have to be reverse complemented on the oligos. Order AP2V6 /5Phos/CACGGGTAGTGTGTATCCTG for amplification.
# AP1V72 = 'GCAAGATTCTCGTCGAGT'  # AP1 directly goes on the probes. Order AP1V6U A*A*GCAAGATTCTCGTCGAG/3deoxyU/  for amplification
# AP2V7 = 'TGTAAGGCACATCTCGGATC' # AP2 will have to be reverse complemented on the oligos. Order AP2V6 /5Phos/TGTAAGGCACATCTCGGATC for amplification.

AP1V8 = 'TAATCTAGCGCGACGTCT'  # AP1 directly goes on the probes. Order AP1V8U T*C*TAATCTAGCGCGACGTCU for amplification
AP2V8 = 'CCACAAGAGGCGCTATGATC' # AP2 will have to be reverse complemented on the oligos. Order AP2V8 /5Phos/CCACAAGAGGCGCTATGATC for amplification.

anchorProbe = 20, rcaPrimer = 18


In [59]:
from itertools import permutations
n_on = 3 # number of on-cycles!
min_len = 152 # minimum length of synthesized probes
non_resolveds = []

# for i, gene in enumerate(allgenes):
for i in barcodes_seq.index:
    gene = barcodes_seq.loc[i, 'Gene name']
    ppd_thisGene = all_ppd.loc[all_ppd['gene_name'] == gene]
    
    if ppd_thisGene.shape[0] == 0:
        print("No probes for {}.".format(gene))
        continue
    
    for ind in ppd_thisGene.index: # for each probe of this gene
        bare_len = (len(AP1V8) + len(anchorProbe) + len(AP2V8) + 
                    len(ppd_thisGene.loc[ind, '7']) + len(ppd_thisGene.loc[ind, '5']) +
                    len(barcodes_seq.loc[i][0]) + len(barcodes_seq.loc[i][1]) + 
                    len(barcodes_seq.loc[i][2]))
        n_Ts = min_len - bare_len
        for j, perm in enumerate(permutations(range(n_on))): # check all permutations of barcodes
            # fp = (AP1V8 +  # Conventional version
            #       ppd_thisGene.loc[ind, '7'] +  
            #       barcodes_seq.loc[i][perm[0]] + 
            #       ''.join(n_Ts * ['A']) + 
            #       anchorProbe + 
            #       barcodes_seq.loc[i][perm[1]] + 
            #       barcodes_seq.loc[i][perm[2]] +
            #       ppd_thisGene.loc[ind, '5'] + 
            #       reverse_complement(AP2V8))

            fp = (AP1V8 +  # SplintR version
                  reverse_complement(ppd_thisGene.loc[ind, '5']) +  # changed to splintR version
                  barcodes_seq.loc[i][perm[0]] + 
                  ''.join(n_Ts * ['A']) + 
                  anchorProbe + 
                  barcodes_seq.loc[i][perm[1]] + 
                  barcodes_seq.loc[i][perm[2]] +
                  reverse_complement(ppd_thisGene.loc[ind, '7']) + # changed to splintR version
                  reverse_complement(AP2V8))
            RNA_target_probe = (reverse_complement(ppd_thisGene.loc[ind, '7']) +  reverse_complement(ppd_thisGene.loc[ind, '5']))
            if not dpnii_site in fp[0:-len(AP2V8)]:
#                 if j > 0:
#                     print("new barcode rescued gene {0}, index {1}".format(gene, ind))
                break                

            elif j == len(list(permutations(range(n_on)))) - 1:
                non_resolveds.append(ind)
                print("No DpnII free barcode for gene {0}, index {1}".format(gene, ind))
        all_ppd.loc[ind, 'full_probe'] = fp
        all_ppd.loc[ind, 'RNA_target_probe'] = RNA_target_probe
            
    all_ppd.loc[all_ppd['gene_name'] == gene, 'barcode'] = barcodes_seq.loc[i]['barcode']     

all_ppd

No DpnII free barcode for gene Col10a1, index 850042
No DpnII free barcode for gene Gadd45g, index 1083083
No probes for Ccl27a.
No probes for Ccl27b.
No probes for Bnip3.
No DpnII free barcode for gene Sox4, index 1070233
No DpnII free barcode for gene Sox4, index 1070235
No probes for H3c6.
No DpnII free barcode for gene Suv39h2, index 111771
No DpnII free barcode for gene Kdm5b, index 68039
No DpnII free barcode for gene Kdm5b, index 68102
No DpnII free barcode for gene Kdm5b, index 68168
No DpnII free barcode for gene Kdm5b, index 68207
No probes for Cbx3.


,0,1,2,3,4,5,6,7,8,9,...,13,gene_name,chr,exon_start,exon_end,probe_center,Annotation,full_probe,RNA_target_probe,barcode
7019,Il17a_chr1_1,chr1:020801129-20801212,0,65.0,65.0,GCAAACATGAGTCCA,50.0,GGGAGAGCTTCATCT,51.0,2.370986,...,1s,Il17a,chr1,020801129,20801212,20801194,CDS,TAATCTAGCGCGACGTCTTGGACTCATGTTTGCAGAACTTGCGCGG...,AGATGAAGCTCTCCCTGGACTCATGTTTGC,00102300
7020,Il17a_chr1_2,chr1:020802320-20802531,0,69.0,69.0,ATCATCCCTCAAAGCT,51.0,CAGCGTGTCCAAACACT,58.0,2.416665,...,2s,Il17a,chr1,020802320,20802531,20802389,CDS,TAATCTAGCGCGACGTCTAGCTTTGAGGGATGATAGAACTTGCGCG...,AGTGTTTGGACACGCTGAGCTTTGAGGGATGAT,00102300
7021,Il17a_chr1_2,chr1:020802320-20802531,0,93.0,93.0,AAACACTGAGGCCA,50.0,AGGACTTCCTCCAGAATGT,58.0,2.437995,...,2s,Il17a,chr1,020802320,20802531,20802413,CDS,TAATCTAGCGCGACGTCTTGGCCTCAGTGTTTAGAACTTGCGCGGA...,ACATTCTGGAGGAAGTCCTTGGCCTCAGTGTTT,00102300
7022,Il17a_chr1_2,chr1:020802320-20802531,0,112.0,112.0,GACTTCCTCCAGAATGT,53.0,GAAGGTCAACCTCAAA,50.0,2.539507,...,2s,Il17a,chr1,020802320,20802531,20802432,CDS,TAATCTAGCGCGACGTCTACATTCTGGAGGAAGTCAGAACTTGCGC...,TTTGAGGTTGACCTTCACATTCTGGAGGAAGTC,00102300
7023,Il17a_chr1_2,chr1:020802320-20802531,0,130.0,130.0,AAGGTCAACCTCAAAGT,53.0,CTTTAACTCCCTTGGC,51.0,2.531687,...,2s,Il17a,chr1,020802320,20802531,20802450,CDS,TAATCTAGCGCGACGTCTACTTTGAGGTTGACCTTAGAACTTGCGC...,GCCAAGGGAGTTAAAGACTTTGAGGTTGACCTT,00102300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1471967,Kdm5d_chrY_26,chrY:000942824-946316,0,1052.0,1052.0,TTGCTGTGGGACTCAA,55.0,CAGCATTTGTTTCTGAA,50.0,2.611300,...,14s,Kdm5d,chrY,000942824,946316,943876,3'UTR,TAATCTAGCGCGACGTCTTTGAGTCCCACAGCAAAACTGCGACCGT...,TTCAGAAACAAATGCTGTTGAGTCCCACAGCAA,00033020
1471968,Kdm5d_chrY_26,chrY:000942824-946316,0,1278.0,1278.0,ACCACTGCCCAACT,53.0,CTATGCTTCTTGATACATG,51.0,2.552558,...,14s,Kdm5d,chrY,000942824,946316,944102,3'UTR,TAATCTAGCGCGACGTCTAGTTGGGCAGTGGTAACTGCGACCGTCG...,CATGTATCAAGAAGCATAGAGTTGGGCAGTGGT,00033020
1471969,Kdm5d_chrY_26,chrY:000942824-946316,0,3086.0,3086.0,TGGAGATGAATAAGTTGT,50.0,AAGGGTTGAGGCTAA,50.0,2.485002,...,14s,Kdm5d,chrY,000942824,946316,945910,3'UTR,TAATCTAGCGCGACGTCTACAACTTATTCATCTCCAAACTGCGACC...,TTAGCCTCAACCCTTACAACTTATTCATCTCCA,00033020
1471970,Kdm5d_chrY_26,chrY:000942824-946316,0,3233.0,3233.0,TCCATGAGATAAGGCA,51.0,ACTGTAAGGCATTTTCT,50.0,2.493049,...,14s,Kdm5d,chrY,000942824,946316,946057,3'UTR,TAATCTAGCGCGACGTCTTGCCTTATCTCATGGAAACTGCGACCGT...,AGAAAATGCCTTACAGTTGCCTTATCTCATGGA,00033020


In [60]:
print('{} genes have no good probes designed'.format(set(allgenes) - set(all_ppd['gene_name'])))
allgenes = set(allgenes) & set(all_ppd['gene_name'])

{'Ccl27a', 'Cbx3', 'Bnip3', 'H3c6', 'Ccl27b'} genes have no good probes designed


### select n_max probes for each gene according to the strategy above
removing the probes that do have DpnII before the selection

In [61]:
# select probes based on the order: CDS, 5'UTR, 3'UTR and prioritize probes close to TSS and non-overlapping probes

dist_for_nonoverlap = 33
selprobes = []
for g in set(allgenes):
# for g in ['Smarca5','Il17a']:
    ppd_g = all_ppd.loc[~all_ppd.index.isin(non_resolveds)].query("gene_name == @g")
    ppd_g = ppd_g.sort_values('probe_center')
    n_probe = ppd_g.shape[0]
    if n_probe <= n_max:
        sel_index = ppd_g.index
        sel_index = sel_index.append(ppd_g.sample(n = n_max - len(sel_index), replace=True).index)
        sel_g = ppd_g.loc[sel_index]
    else:
        sel_index = []
        sel_probe_center = []
        while len(sel_index) < n_max:
            if ppd_g['10'].iloc[0] == 'W': # Walson strand or + strand, select probes close to TSS - smaller probe_center
                for annot in ['CDS', "5'UTR", "3'UTR"]:
                    if len(ppd_g.loc[ppd_g['Annotation'] == annot]) != 0 and len(sel_index) == 0:
                        sel_index.append(ppd_g.loc[ppd_g['Annotation'] == annot].index[0])
                        sel_probe_center.append(ppd_g.loc[ppd_g['Annotation'] == annot]['probe_center'].iloc[0])
                        ppd_g = ppd_g.drop(ppd_g.loc[ppd_g['Annotation'] == annot].index[0])
                    if len(sel_index) < n_max:
                        for index in ppd_g.loc[ppd_g['Annotation'] == annot].index:
                            probe_center = ppd_g.loc[index, 'probe_center']
                            if all(np.abs(probe_center - np.array(sel_probe_center)) > dist_for_nonoverlap):
                                sel_index.append(index)
                                sel_probe_center.append(probe_center)
                                ppd_g = ppd_g.drop(index)
                            if len(sel_index) == n_max:
                                break
                if len(sel_index) < n_max:
                    sel_index.extend(ppd_g.sample(n = n_max - len(sel_index), replace=True).index.values)
            else: # Crick strand or - strand, select probes close to TSS - larger probe_center
                for annot in ['CDS', "5'UTR", "3'UTR"]:
                    if len(ppd_g.loc[ppd_g['Annotation'] == annot]) != 0 and len(sel_index) == 0:
                        sel_index.append(ppd_g.loc[ppd_g['Annotation'] == annot].index[-1])
                        sel_probe_center.append(ppd_g.loc[ppd_g['Annotation'] == annot]['probe_center'].iloc[-1])
                        ppd_g = ppd_g.drop(ppd_g.loc[ppd_g['Annotation'] == annot].index[-1])
                    if len(sel_index) < n_max:
                        for index in ppd_g.loc[ppd_g['Annotation'] == annot].index[::-1]:
                            probe_center = ppd_g.loc[index, 'probe_center']
                            if all(np.abs(probe_center - np.array(sel_probe_center)) > dist_for_nonoverlap):
                                sel_index.append(index)
                                sel_probe_center.append(probe_center)
                                ppd_g = ppd_g.drop(index)
                            if len(sel_index) == n_max:
                                break
                if len(sel_index) < n_max:
                    sel_index.extend(ppd_g.sample(n = n_max - len(sel_index), replace=True).index.values)
        ppd_g = all_ppd.loc[~all_ppd.index.isin(non_resolveds)].query("gene_name == @g")
        sel_g = ppd_g.loc[sel_index]
    selprobes.append(sel_g)
selprobes = pd.concat(selprobes)
print("{} probes are duplicated".format(selprobes.shape[0] - selprobes.index.unique().shape[0]))
selprobes


67 probes are duplicated


,0,1,2,3,4,5,6,7,8,9,...,13,gene_name,chr,exon_start,exon_end,probe_center,Annotation,full_probe,RNA_target_probe,barcode
205745,Dnmt3b_chr2_3,chr2:153503345-153503522,0,18.0,18.0,AACAATGAAGGGAGACA,52.0,GCAGACATCTGAATGA,50.0,2.581573,...,1s,Dnmt3b,chr2,153503345,153503522,153503363,CDS,TAATCTAGCGCGACGTCTTGTCTCCCTTCATTGTTTCGTACTTCGA...,TCATTCAGATGTCTGCTGTCTCCCTTCATTGTT,00020203
205746,Dnmt3b_chr2_3,chr2:153503345-153503522,0,74.0,74.0,AGGAGTGCATTATCGTT,53.0,AATGGGAACTTCAGTG,50.0,2.600552,...,1s,Dnmt3b,chr2,153503345,153503522,153503419,CDS,TAATCTAGCGCGACGTCTAACGATAATGCACTCCTTCGTACTTCGA...,CACTGAAGTTCCCATTAACGATAATGCACTCCT,00020203
205749,Dnmt3b_chr2_3,chr2:153503345-153503522,0,142.0,142.0,AGTCTTGGAGGCAAT,51.0,CTGCACAGAGCCAGTCT,59.0,2.263230,...,1s,Dnmt3b,chr2,153503345,153503522,153503487,CDS,TAATCTAGCGCGACGTCTATTGCCTCCAAGACTTCGTACTTCGACG...,AGACTGGCTCTGTGCAGATTGCCTCCAAGACT,00020203
205751,Dnmt3b_chr2_4,chr2:153504063-153504124,0,38.0,38.0,TCTAAGAGGGAGGTCT,51.0,CCAGCCTTCTGAATTA,50.0,2.487027,...,0s,Dnmt3b,chr2,153504063,153504124,153504101,CDS,TAATCTAGCGCGACGTCTAGACCTCCCTCTTAGAACGTCTGCGTAC...,TAATTCAGAAGGCTGGAGACCTCCCTCTTAGA,00020203
205752,Dnmt3b_chr2_5,chr2:153504634-153504753,0,16.0,16.0,ACATGACAGGAGATGG,52.0,AGACAGAGATGATGAAG,50.0,2.532911,...,0s,Dnmt3b,chr2,153504634,153504753,153504650,CDS,TAATCTAGCGCGACGTCTCCATCTCCTGTCATGTTCGTACTTCGAC...,CTTCATCATCTCTGTCTCCATCTCCTGTCATGT,00020203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479060,Col1a2_chr6_7,chr6:004515210-4515254,0,17.0,17.0,TTTAATGGGACCCAGA,51.0,GGCCCTCCTGGTGC,59.0,1.832950,...,0s,Col1a2,chr6,004515210,4515254,4515227,CDS,TAATCTAGCGCGACGTCTTCTGGGTCCCATTAAACGTATCGGTAGT...,GCACCAGGAGGGCCTCTGGGTCCCATTAAA,20000021
479061,Col1a2_chr6_8,chr6:004515481-4515534,0,20.0,20.0,TCAAGGTTTCCAAGGA,51.0,CCTGCTGGTGAACCT,55.0,2.220026,...,1s,Col1a2,chr6,004515481,4515534,4515501,CDS,TAATCTAGCGCGACGTCTTCCTTGGAAACCTTGACGTATCGGTAGT...,AGGTTCACCAGCAGGTCCTTGGAAACCTTGA,20000021
479062,Col1a2_chr6_9,chr6:004515629-4515682,0,35.0,35.0,CAGCTGGCTCTCCT,54.0,GGCAAGGCTGGTGAG,57.0,1.380224,...,0s,Col1a2,chr6,004515629,4515682,4515664,CDS,TAATCTAGCGCGACGTCTAGGAGAGCCAGCTGCGTATCGGTAGTCG...,CTCACCAGCCTTGCCAGGAGAGCCAGCTG,20000021
479063,Col1a2_chr6_10,chr6:004515961-4516014,0,21.0,21.0,TGGAAAACCCGGAA,50.0,GACCTGGGGAGAGAGGA,59.0,2.100101,...,0s,Col1a2,chr6,004515961,4516014,4515982,CDS,TAATCTAGCGCGACGTCTTTCCGGGTTTTCCACGTATCGGTAGTCG...,TCCTCTCTCCCCAGGTCTTCCGGGTTTTCCA,20000021


In [62]:
# selprobes = []
# for g in set(allgenes) - set(['Ccl27a','Cbx3','Bnip3']):
#     ppd_g = all_ppd.loc[~all_ppd.index.isin(non_resolveds)].query("gene_name == @g")
#     ppd_g_GGGG_free = ppd_g.loc[(~ppd_g['7'].str.contains('CCCC')) & (~ppd_g['5'].str.contains('CCCC'))]
#     n_l = (ppd_g_GGGG_free['subset'] == 'l').sum()
#     n_s = (ppd_g_GGGG_free['subset'] == 's').sum()
#     if n_l > n_max: # too many non-overlapping probes
#         sel_g = ppd_g_GGGG_free.query("subset == 'l'").sample(n = n_max, replace=False)
#     elif n_l + n_s > n_max: # select all of l-subset and fill up with s-subset
#         s_index = ppd_g_GGGG_free.query("subset == 's'").sample(n = n_max-n_l, replace=False).index
#         sel_index = ppd_g_GGGG_free.query("subset=='l'").index.append(s_index)
#         sel_g = ppd_g_GGGG_free.loc[sel_index]
#     else: # less than n_max probes 
#         sel_index = ppd_g_GGGG_free.index
#         sel_index = sel_index.append(ppd_g_GGGG_free.sample(n = n_max - len(sel_index), replace=True).index)
#         sel_g = ppd_g_GGGG_free.loc[sel_index]
#     selprobes.append(sel_g)
# selprobes = pd.concat(selprobes)
# print("{} probes are duplicated".format(selprobes.shape[0] - selprobes.index.unique().shape[0]))
# selprobes

In [62]:
selprobes['Annotation'].value_counts()

Annotation
CDS      2734
3'UTR     202
5'UTR     124
Name: count, dtype: int64

#### writing the probes to file in .ppd format

In [63]:
unResolved = all_ppd['full_probe'].str.slice(0, -len(AP2V8)).str.contains(dpnii_site)
unResolved = all_ppd.loc[unResolved]
unResolved[['gene_name', 'barcode', 'full_probe']].to_csv('2408_MCrossTissue_probesWithGATC_3col.tsv', sep = '\t', index = False)
all_ppd = all_ppd.drop(unResolved.index)

In [64]:
selprobes.values[10]

array(['Hs6st3_chr14_1', 'chr14:119375753-119376530', 0, 78.0, 78.0,
       'AACGGGACCATGGA', 53.0, 'TGAAAGGTTCAACAAGT', 51.0, 2.542873, 'W',
       'AACGGGACCATGGATGAAAGGTTCAACAAGT', 0.1, '0s', 'Hs6st3', 'chr14',
       '119375753', '119376530', 119375831, 'CDS',
       'TAATCTAGCGCGACGTCTTCCATGGTCCCGTTCGTATCGGTAGTCGCAACGCAAACTTCAGCTTCCCGATATCCGGACGAACGGTCGAGATTTACAACTGCGACCGTCGGCTTACACTTGTTGAACCTTTCAGATCATAGCGCCTCTTGTGG',
       'ACTTGTTGAACCTTTCATCCATGGTCCCGTT', '20330000'], dtype=object)

### add "A" to the 5' to get them to ```target_len``` length

In [65]:
selprobes['full_probe'] = selprobes['full_probe'].str.pad(target_len, side='left', fillchar='A')
selprobes['full_probe'].head()

205745    TAATCTAGCGCGACGTCTTGTCTCCCTTCATTGTTTCGTACTTCGA...
205746    TAATCTAGCGCGACGTCTAACGATAATGCACTCCTTCGTACTTCGA...
205749    TAATCTAGCGCGACGTCTATTGCCTCCAAGACTTCGTACTTCGACG...
205751    TAATCTAGCGCGACGTCTAGACCTCCCTCTTAGAACGTCTGCGTAC...
205752    TAATCTAGCGCGACGTCTCCATCTCCTGTCATGTTCGTACTTCGAC...
Name: full_probe, dtype: object

In [66]:
# forming the fasta name : gene_name + barcode + probe#
selprobes['fasta_name'] = (selprobes['gene_name'] + '_' + 
                            selprobes['barcode'] + '_' +
                            (selprobes.groupby('barcode').cumcount() + 1).astype(str))
# writing the fasta name:
with open("2408_MCrossTissue_probes_3col_V8.fa", 'w') as writer:
    for i, row in selprobes.iterrows():
        writer.write('>{0}\n'.format(row['fasta_name'])) # write the name
        writer.write("{0}\n".format(row['full_probe']))

selprobes[['fasta_name', 'full_probe']].to_csv("2408_MCrossTissue_probes_3col_V8_summary.tsv", sep = '\t', index = False)
selprobes.to_csv("2408_MCrossTissue_probes_3col_V8_full.ppd", sep = '\t', index = False)
selprobes.head()

,0,1,2,3,4,5,6,7,8,9,...,gene_name,chr,exon_start,exon_end,probe_center,Annotation,full_probe,RNA_target_probe,barcode,fasta_name
205745,Dnmt3b_chr2_3,chr2:153503345-153503522,0,18.0,18.0,AACAATGAAGGGAGACA,52.0,GCAGACATCTGAATGA,50.0,2.581573,...,Dnmt3b,chr2,153503345,153503522,153503363,CDS,TAATCTAGCGCGACGTCTTGTCTCCCTTCATTGTTTCGTACTTCGA...,TCATTCAGATGTCTGCTGTCTCCCTTCATTGTT,00020203,Dnmt3b_00020203_1
205746,Dnmt3b_chr2_3,chr2:153503345-153503522,0,74.0,74.0,AGGAGTGCATTATCGTT,53.0,AATGGGAACTTCAGTG,50.0,2.600552,...,Dnmt3b,chr2,153503345,153503522,153503419,CDS,TAATCTAGCGCGACGTCTAACGATAATGCACTCCTTCGTACTTCGA...,CACTGAAGTTCCCATTAACGATAATGCACTCCT,00020203,Dnmt3b_00020203_2
205749,Dnmt3b_chr2_3,chr2:153503345-153503522,0,142.0,142.0,AGTCTTGGAGGCAAT,51.0,CTGCACAGAGCCAGTCT,59.0,2.263230,...,Dnmt3b,chr2,153503345,153503522,153503487,CDS,TAATCTAGCGCGACGTCTATTGCCTCCAAGACTTCGTACTTCGACG...,AGACTGGCTCTGTGCAGATTGCCTCCAAGACT,00020203,Dnmt3b_00020203_3
205751,Dnmt3b_chr2_4,chr2:153504063-153504124,0,38.0,38.0,TCTAAGAGGGAGGTCT,51.0,CCAGCCTTCTGAATTA,50.0,2.487027,...,Dnmt3b,chr2,153504063,153504124,153504101,CDS,TAATCTAGCGCGACGTCTAGACCTCCCTCTTAGAACGTCTGCGTAC...,TAATTCAGAAGGCTGGAGACCTCCCTCTTAGA,00020203,Dnmt3b_00020203_4
205752,Dnmt3b_chr2_5,chr2:153504634-153504753,0,16.0,16.0,ACATGACAGGAGATGG,52.0,AGACAGAGATGATGAAG,50.0,2.532911,...,Dnmt3b,chr2,153504634,153504753,153504650,CDS,TAATCTAGCGCGACGTCTCCATCTCCTGTCATGTTCGTACTTCGAC...,CTTCATCATCTCTGTCTCCATCTCCTGTCATGT,00020203,Dnmt3b_00020203_5


In [67]:
codebook = (selprobes['gene_name'] + '_' + selprobes['barcode']).unique()
with open("./2408_MCrossTissue_codebook_3col.txt", "w") as writer:
    for c in codebook:
        writer.writelines("{}\n".format(c))
Empty_codebook = empty_df['Gene name'] + '_' + empty_df.iloc[:,1:8].astype(str).agg(''.join, axis=1)
with open("./2408_MCrossTissue_codebook_3col.txt", "a") as writer:
    for c in Empty_codebook:
        writer.writelines("{}\n".format(c))

### Making sure that the combination of these barcodes is recoverable 

In [38]:
import os, pandas as pd, numpy as np

def onehot(bc):
    bc = list(bc)
    oh = []
    for char in bc:
        if char == '1':
            oh += [1, 0, 0]
        elif char == '2': 
            oh += [0, 1, 0]
        elif char == '3':
            oh += [0, 0, 1]
        else:
            oh += [0, 0, 0]
    return np.array(oh)

cb_file = "./short_genes_codebook.txt"
codebook = pd.read_csv(cb_file, sep="_", header=None, names=['gene', 'barcode'], dtype=str)
codebook['onehot'] = codebook['barcode'].apply(onehot)
display(codebook)

from itertools import combinations

mod2_dict = {}
nomod_dict = {}
for i, (g1, g2) in enumerate(combinations(codebook['gene'], 2)):
    bc1 = codebook.query('gene==@g1')['onehot'].iloc[0]
    bc2 = codebook.query('gene==@g2')['onehot'].iloc[0]
    nomod = bc1 + bc2
    mod2 = (nomod>0).astype(int)
    nomod_key = ''.join(nomod.astype(str))
    mod2_key = ''.join(mod2.astype(str))
    if mod2_key in mod2_dict:
        mod2_dict[mod2_key] += [(g1, g2)]
    else:
        mod2_dict[mod2_key] = [(g1, g2)]
    if nomod_key in nomod_dict:
        nomod_dict[nomod_key] += [(g1, g2)]
    else:
        nomod_dict[nomod_key] = [(g1, g2)]
    
    if i%1000 == 0:
        print(i)
        
mod2_counts = {key: len(val) for key, val in mod2_dict.items()}
mod2_df = pd.DataFrame.from_dict(mod2_counts, orient='index', columns=['count'])
mod2_df['gene_pairs'] = [val for key, val in mod2_dict.items()]
mod2_df = mod2_df.sort_values(by='count', ascending=False)
display(mod2_df)

nomod_counts = {key: len(val) for key, val in nomod_dict.items()}
nomod_df = pd.DataFrame.from_dict(nomod_counts, orient='index', columns=['count'])
nomod_df['gene_pairs'] = [val for key, val in nomod_dict.items()]
nomod_df = nomod_df.sort_values(by='count', ascending=False)
display(nomod_df)


print(len(list(combinations(codebook['gene'], 2))))
display(mod2_df['count'].value_counts())

display(nomod_df['count'].value_counts())

FileNotFoundError: [Errno 2] No such file or directory: './short_genes_codebook.txt'

In [ ]:
bc1 = codebook.query('gene=="LINC02218"')['onehot'].iloc[0]
bc2 = codebook.query('gene=="CMTM8"')['onehot'].iloc[0]
nomod = bc1 + bc2
mod2 = (nomod>0).astype(int)
nomod_key = ''.join(nomod.astype(str))
mod2_key = ''.join(mod2.astype(str))
# mod2_df[mod2_key]
print(mod2_key)
display(mod2_df.loc[mod2_key])
print(nomod_key)
display(nomod_df.loc[nomod_key])

In [ ]:
for i, pairlist in enumerate(mod2_df['gene_pairs']):
    for pair in pairlist:
        if 'CMTM8' in pair:
            n = mod2_df.iloc[i, 0]
            if n > 1:
                print(pairlist)
                break


In [ ]:
for i, pairlist in enumerate(mod2_df['gene_pairs']):
    for pair in pairlist:
        if 'TESC' in pair:
            n = mod2_df.iloc[i, 0]
            if n > 1:
                print(pairlist)
        break


In [ ]:
import os, pandas as pd, numpy as np

def onehot(bc):
    bc = list(bc)
    oh = []
    for char in bc:
        if char == '1':
            oh += [1, 0, 0]
        elif char == '2': 
            oh += [0, 1, 0]
        elif char == '3':
            oh += [0, 0, 1]
        else:
            oh += [0, 0, 0]
    return np.array(oh)

cb_file = "./short_genes_codebook.txt"
codebook = pd.read_csv(cb_file, sep="_", header=None, names=['gene', 'barcode'], dtype=str)
codebook['onehot'] = codebook['barcode'].apply(onehot)
display(codebook)

from itertools import combinations

mod2_dict = {}
nomod_dict = {}
for i, (g1, g2, g3) in enumerate(combinations(codebook['gene'], 3)):
    bc1 = codebook.query('gene==@g1')['onehot'].iloc[0]
    bc2 = codebook.query('gene==@g2')['onehot'].iloc[0]
    bc3 = codebook.query('gene==@g3')['onehot'].iloc[0]    
    nomod = bc1 + bc2 + bc3
    mod2 = (nomod>0).astype(int)
    nomod_key = ''.join(nomod.astype(str))
    mod2_key = ''.join(mod2.astype(str))
    if mod2_key in mod2_dict:
        mod2_dict[mod2_key] += [(g1, g2, g3)]
    else:
        mod2_dict[mod2_key] = [(g1, g2, g3)]
    if nomod_key in nomod_dict:
        nomod_dict[nomod_key] += [(g1, g2, g3)]
    else:
        nomod_dict[nomod_key] = [(g1, g2, g3)]
    
    if i%1000 == 0:
        print(i)
        
mod2_counts = {key: len(val) for key, val in mod2_dict.items()}
mod2_df = pd.DataFrame.from_dict(mod2_counts, orient='index', columns=['count'])
mod2_df['gene_pairs'] = [val for key, val in mod2_dict.items()]
mod2_df = mod2_df.sort_values(by='count', ascending=False)
display(mod2_df)

nomod_counts = {key: len(val) for key, val in nomod_dict.items()}
nomod_df = pd.DataFrame.from_dict(nomod_counts, orient='index', columns=['count'])
nomod_df['gene_pairs'] = [val for key, val in nomod_dict.items()]
nomod_df = nomod_df.sort_values(by='count', ascending=False)
display(nomod_df)


print(len(list(combinations(codebook['gene'], 3))))
display(mod2_df['count'].value_counts())

display(nomod_df['count'].value_counts())